# Week8 Homework
## Working with Strings

The data frame contains notes taken by staff members. Your task is to clean it up to make it usable for analysis. 

### Tasks
* Create start date, end date columns and populate the values (If notes don't include month or day use 1 for missing value) 

i.e: 
    
    - 2018 => 2018-01-01  
    - June 2021 => 2021-06-01  
    - 2020 => 2020-01-01

* Create phone number column (If notes don't include phone numbers, use None)
* Create a column `duration` showing difference between start & end dates in years as a whole number

Note: Choose one of the following datasets to work on based on your taste! 

In [1]:
import pandas as pd
from datetime import datetime
import re
import numpy as np

In [2]:
spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_spicy.csv'
less_spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_less_spicy.csv'

In [3]:
df = pd.read_csv(less_spicy, index_col=0)

In [4]:
df['notes']

0                          start July 1970/end 2013-06-17
1                            start 1971-02 end 04/10/2012
2       start January 1977 end 12/20/2018 pn: 001-464-...
3                         start 1986-02 end February 2010
4                               start 1975/end 07/27/2018
                              ...                        
1995                      start 05/04/1988-end 08/19/2018
1996             start 1978-08-end 2017-06-p#  1886569390
1997                      start 05/06/1984/end 08/11/2021
1998     start 2004-02-22 end 2018 p/n  709.702.0599x5879
1999                      start 1977-01-31 end 05/18/2020
Name: notes, Length: 2000, dtype: object

In [5]:
df['start_date'] = df['notes'].str.extract(r'\bstart(.+?)end\b', expand=False).str.rstrip("/").str.rstrip("-").str.strip()

In [6]:
df['end_date'] = df.notes.apply(lambda x: re.findall(r"\bend.+pn\b|\bend.+P\b|\bend.+p\/|\bend.+#|\bend.+\b", x)[0].strip("end").lower().rstrip("/p").rstrip("p").strip("-p#").strip("-p/").strip())

In [7]:
def getPhoneNum(x):
    match=re.search(r"\d{0,3}?\W\d\d\d\W\d\d\d\W\d\d\d\d([x]?\d{3,5})?|\d{10,}", x)
    return match.group() if match else None


def parsing_date(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y', '%B %Y', '%Y', '%Y-%m', '%m/%d/%Y'):
        try:
            d = datetime.strptime(text, fmt)
            return d.strftime('%Y-%m-%d')
        except ValueError:
            pass
    raise ValueError('no valid date format found', text)

In [8]:
df['phone_num']=df.notes.apply(lambda x: getPhoneNum(x))

In [9]:
df['start_date']=df.start_date.apply(parsing_date)
df['start_date']=pd.to_datetime(df['start_date'], format='%Y-%m-%d', errors = 'coerce')

In [10]:
df['end_date']=df.end_date.apply(parsing_date)
df['end_date']=pd.to_datetime(df['end_date'], format='%Y-%m-%d', errors = 'coerce')

In [11]:
df['duration']=(df['end_date']-df['start_date'])/np.timedelta64(1, 'Y')
df['duration']=df['duration'].apply(np.floor)
df.head(20)

,email,notes,start_date,end_date,phone_num,duration
0,margaret41@santos.info,start July 1970/end 2013-06-17,1970-07-01,2013-06-17,None,42.0
1,donna16@jimenez.com,start 1971-02 end 04/10/2012,1971-02-01,2012-10-04,None,41.0
2,edwardslinda@cabrera-murphy.org,start January 1977 end 12/20/2018 pn: 001-464-...,1977-01-01,2018-12-20,001-464-616-8819,41.0
3,gutierrezdenise@hotmail.com,start 1986-02 end February 2010,1986-02-01,2010-02-01,None,24.0
4,jherrera@koch.com,start 1975/end 07/27/2018,1975-01-01,2018-07-27,None,43.0
5,meganmcdonald@wallace.info,start January 1996/end May 2017,1996-01-01,2017-05-01,None,21.0
6,martinkellie@thomas-cannon.net,start 2002-08-03-end 09/11/2016,2002-08-03,2016-11-09,None,14.0
7,elizabethmoore@hotmail.com,start 1978/end 2020-09,1978-01-01,2020-09-01,None,42.0
8,rosemarcus@yahoo.com,start 2008-11-30/end 2011-12/P# 001-467-263-22...,2008-11-30,2011-12-01,001-467-263-2241x61560,3.0
9,jessicacosta@henson.com,start 2003-11 end February 2018 P# +1-946-742-...,2003-11-01,2018-02-01,1-946-742-6723x151,14.0


<h5>Reference</h5>
https://stackoverflow.com/questions/23581128/how-to-format-date-string-via-multiple-formats-in-python
<br>
https://stackoverflow.com/questions/42980662/convert-string-with-month-name-to-datetime